In [2]:
import cv2
import numpy as np

In [3]:
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

classes = []
with open("coco.names", "r") as f:
    lines = f.readlines()
    for line in lines:
        classes.append(line.strip())
    # classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers = [layer_names[i-1] for i in net.getUnconnectedOutLayers()]

colors = np.random.uniform(0, 255, size=(len(classes), 3))

### Obect detection from a photo
1. Load in image
2. Detect objects

In [6]:
img = cv2.imread("sample.jpg")

height, width, channels = img.shape

In [7]:
# Detecting objects
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), swapRB=True, crop=False)
net.setInput(blob) # input image into algorithm network
results = net.forward(output_layers) # detected results

class_ids = []
confidences = []
boxes = []

for result in results:
    for detection in result:
        scores = detection[5:]
        class_id = np.argmax(scores) 
        confidence = scores[class_id]
        
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4) # Non Maximum Suppresion   

font = cv2.FONT_HERSHEY_COMPLEX_SMALL

for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[i]
        caption = f"{label}: {confidence}"
        cv2.rectangle(img, (x, y), (x + w, y + h), color, thickness = 2)
        cv2.putText(img, caption, (x, y + 30), font, 3, color, 3)
        
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Obect detection from a video (webcam)
1. Launch webcam
2. Detect objects

In [8]:
cap = cv2.VideoCapture("face_video.mp4")
# parameter > 0 : webcam
# parameter > filename : load in video
cap.set(3,640)
cap.set(4,480)

False

In [9]:
while True:
    ret, img = cap.read()
    widths = img.shape[0]
    height = img.shape[1]
    
    if ret:
        # Detecting objects
        blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), swapRB=True, crop=False)
        net.setInput(blob) # input image into algorithm network
        results = net.forward(output_layers) # detected results

        class_ids = []
        confidences = []
        boxes = []
        
        for result in results:
            for detection in result:
                scores = detection[5:]
                class_id = np.argmax(scores) 
                confidence = scores[class_id]
                
                if confidence > 0.5:
                    # Object detected
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)
                    
        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
        # Non Maximum Suppresion

        font = cv2.FONT_HERSHEY_COMPLEX_SMALL

        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                color = colors[i]
                
                caption = f"{label}: {confidence}"
                cv2.rectangle(img, (x, y), (x + w, y + h), color, thickness = 2)
                cv2.putText(img, caption, (x, y + 30), font, 3, color, 3)
        
        cv2.imshow("Image", img)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
    
cap.release()
cv2.destroyAllWindows()